# Analysis of "Azerbaijani Laundromat" transaction data

## Set up the environment

import the Python library dependencies

In [1]:
from collections import defaultdict
from datetime import datetime, timedelta
import itertools
import pathlib
import statistics
import typing

from icecream import ic
import networkx as nx
import pandas as pd
import pyvis

In [2]:
pd.set_option("display.max_rows", None)

## Load the dataset

load the OCCRP data for the "Azerbaijani Laundromat" leaked bank transactions <https://github.com/cj2001/senzing_occrp_mapping_demo/blob/main/occrp_17k.csv>

In [3]:
occrp_file: pathlib.Path = pathlib.Path("occrp_17k.csv")
df_orig: pd.DataFrame = pd.read_csv(occrp_file)

In [4]:
df_orig.head()

,payer_name,payer_jurisdiction,payer_account,source_file,amount_orig,id,beneficiary_type,beneficiary_core,amount_orig_currency,beneficiary_name,...,beneficiary_name_norm,payer_core,beneficiary_account,purpose,date,amount_usd,amount_eur,payer_type,payer_name_norm,payer_bank_country
0,AZARBAYCAN METANOL KOMPANI MMC,AZ,33817018409333311204,pdf/LCM ALLIANCE Account statement 30.06.12-31...,535470.00,6049,Company,True,USD,LCM ALLIANCE LLP,...,LCM ALLIANCE LLP,False,EE27 3300 3335 0561 0002,1206295100052180 OCT4121800021 ADVANCE PAYM FO...,2012-06-30,535470,"$431,762.31",Company,AZARBAYCAN METANOL KOMPANI MMC,33
1,LCM ALLIANCE LLP,GB,EE27 3300 3335 0561 0002,pdf/LCM ALLIANCE Account statement 30.06.12-31...,-535000.00,6050,Company,False,USD,MOBILA LLP,...,MOBILA LLP,True,333504500003,1207035026699176 INVOICE.No 6215 DD 25.05.2012,2012-07-03,535000,"$423,688.44",Company,LCM ALLIANCE LLP,EE
2,SKN ELECTRICAL SERVICES LIMITEDACCO,GB,20100374548222,pdf/METASTAR Account statement 30.06.12-31.12....,90535.19,10623,Company,True,USD,METASTAR INVEST LLP,...,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,"1207065103089249 /FEE/USD4,81",2012-07-06,90536,"$71,698.53",Company,SKN ELECTRICAL SERVICES LIMITED,20
3,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-90520.00,15589,Company,False,USD,INMAXO CAPITAL CORP,...,INMAXO CAPITAL CORP.,True,333455870002,1207095022358525 DOGOVOR ZAYMA,2012-07-09,90520,"$71,686.50",Company,METASTAR INVEST LLP,EE
4,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-60.00,10624,Company,False,USD,INMAXO CAPITAL CORP,...,INMAXO CAPITAL CORP.,True,333455870002,1207135024578077 DOGOVOR ZAYMA,2012-07-13,60,$47.52,Company,METASTAR INVEST LLP,EE


In [5]:
df_orig.columns

Index(['payer_name', 'payer_jurisdiction', 'payer_account', 'source_file',
       'amount_orig', 'id', 'beneficiary_type', 'beneficiary_core',
       'amount_orig_currency', 'beneficiary_name', 'beneficiary_jurisdiction',
       'investigation', 'beneficiary_bank_country', 'beneficiary_name_norm',
       'payer_core', 'beneficiary_account', 'purpose', 'date', 'amount_usd',
       'amount_eur', 'payer_type', 'payer_name_norm', 'payer_bank_country'],
      dtype='object')

reduce the dataframe to just the slice needed for this analysis

In [6]:
cols_keep: typing.List[ str ] = [
    "date",
    "amount_orig",
    "payer_name",
    "beneficiary_name",
    "purpose",
    ]

diff: typing.Set[ str ] = set(df_orig.columns) - set(cols_keep)
    
df: pd.DataFrame = df_orig.drop(
    diff,
    axis = 1,
    inplace = False,
).sort_values(by = "date").reindex(columns = cols_keep)

df = df[df["amount_orig"] >= 0]
df.date = pd.to_datetime(df.date)

df.head()

,date,amount_orig,payer_name,beneficiary_name,purpose
0,2012-06-30,535470.00,AZARBAYCAN METANOL KOMPANI MMC,LCM ALLIANCE LLP,1206295100052180 OCT4121800021 ADVANCE PAYM FO...
2,2012-07-06,90535.19,SKN ELECTRICAL SERVICES LIMITEDACCO,METASTAR INVEST LLP,"1207065103089249 /FEE/USD4,81"
5,2012-07-13,189352.15,INMAXO CAPITAL CORP.,METASTAR INVEST LLP,CONTRACT N 001 DD 24.12.2009
6,2012-07-13,225850.00,INMAXO CAPITAL CORP.,METASTAR INVEST LLP,1207135026233261 CONTRACT N 001 DD 24.12.2009
21,2012-07-16,150000.00,MOBILA LLP,METASTAR INVEST LLP,1207165026198353 INVOICE.No 69 DD 13.07.2012


In [7]:
azeri_file: pathlib.Path = pathlib.Path("azeri.csv")
df.to_csv(azeri_file, sep = "\t", encoding = "utf-8")

## Descriptive statistics

In [8]:
df.describe(include = "all").loc[[ "count", "mean", "min", "max", "std", "freq", "unique", ]]

,date,amount_orig,payer_name,beneficiary_name,purpose
count,4069,4.069000e+03,4069,4069,4069
mean,2013-06-24 13:09:11.191939072,7.512028e+05,NaN,NaN,NaN
min,2012-06-30 00:00:00,6.600000e-01,NaN,NaN,NaN
max,2014-12-31 00:00:00,2.055997e+07,NaN,NaN,NaN
std,NaN,1.206048e+06,NaN,NaN,NaN
freq,NaN,NaN,530,1708,37
unique,NaN,NaN,437,4,3781


## Network analysis

In [9]:
net: pyvis.network.Network = pyvis.network.Network(
    notebook = True,
    cdn_resources = "in_line",
    height = "700px", 
    width = "100%",
)

graph: nx.DiGraph = nx.DiGraph()

edge_xact: dict = defaultdict(list)

In [10]:
node_names: list = []

for index, row in df.iterrows():
    src_label: str = row["payer_name"]
    dst_label: str = row["beneficiary_name"]

    if src_label not in node_names:
        node_names.append(src_label)
        src_id: int = node_names.index(src_label)

        net.add_node(
            src_id,
            label = src_label,
        )
        graph.add_node(
            src_id,
            name = src_label,
        )            
    else:
        src_id = node_names.index(src_label)


    if dst_label not in node_names:
        node_names.append(dst_label)
        dst_id: int = node_names.index(dst_label)

        net.add_node(
            dst_id,
            label = dst_label,
        )

        graph.add_node(
            dst_id,
            name = dst_label,
        )
    else:
        dst_id = node_names.index(dst_label)

    net.add_edge(
        src_id,
        dst_id,
    )

    graph.add_edge(
        src_id,
        dst_id,
    )

    edge_xact[ (src_id, dst_id) ].append({
        "amount": row["amount_orig"],
        "date": datetime.fromisoformat(str(row["date"])),
    })    

visualize the network of shell corps involved

In [11]:
net.force_atlas_2based()
net.show_buttons(filter_ = ["physics"])

#net.toggle_physics(False)
net.show("occrp.html")

occrp.html


describe the distributions of transaction inter-arrival times and amounts

In [12]:
summary_data: list = []

for edge, dat in edge_xact.items():
    if len(dat) > 1:
        dates: list = sorted([
            xact["date"]
            for xact in dat
        ], reverse = False)

        inter_arrival: list = [
            (pair[1] - pair[0]).days
            for pair in itertools.pairwise(dates)
        ]

        amounts: list = [
            xact["amount"]
            for xact in dat
        ]

        summary_data.append({
            "src_id": edge[0],
            "dst_id": edge[1],
            "median_amount": statistics.median(amounts),
            "total_amount": sum(amounts),
            "inter_arrival": statistics.mean(inter_arrival),
        })

df_summary: pd.DataFrame = pd.DataFrame.from_dict(
    summary_data,
    orient = "columns"
)

df_summary.head()

,src_id,dst_id,median_amount,total_amount,inter_arrival
0,0,1,109510.00,1511916.50,39.750000
1,2,3,10101.00,110063.19,59.500000
2,4,3,189352.15,415231.08,1.500000
3,5,3,300000.00,4729714.00,1.916667
4,6,3,50000.00,12747647.00,3.629310


In [13]:
df_summary.describe()

,src_id,dst_id,median_amount,total_amount,inter_arrival
count,252.000000,252.000000,2.520000e+02,2.520000e+02,252.000000
mean,173.626984,40.829365,3.798608e+05,1.161657e+07,21.156845
std,126.250130,101.164882,5.301957e+05,8.014517e+07,32.745006
min,0.000000,1.000000,1.100000e+02,2.200000e+02,0.000000
25%,64.750000,1.000000,6.717762e+04,3.476025e+05,3.000000
50%,157.500000,1.000000,1.963890e+05,1.408894e+06,8.708333
75%,268.000000,3.000000,4.762237e+05,4.850007e+06,25.569444
max,433.000000,302.000000,4.000000e+06,1.239991e+09,211.000000
